#Predicting sentiment from product reviews

#Fire up GraphLab Create

In [1]:
import graphlab

#Read some product review data

Loading reviews for a set of baby products. 

In [2]:
products=graphlab.SFrame('amazon_baby.gl/')

This non-commercial license of GraphLab Create for academic use is assigned to partha.cs91@gmail.com and will expire on June 16, 2018.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\PARTHA~1\AppData\Local\Temp\graphlab_server_1502312045.log.0


#Let's explore this data together

Data includes the product name, the review text and the rating of the review. 

In [6]:
#ignore all 3* reviews
products = products[products['rating'] != 3]

In [7]:
#products.head()
products.head()

name,review,rating
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",I love this journal andour nanny uses it ...,4.0


In [8]:
selected_word=['awesome', 'great', 'fantastic', 'amazing', 'love', 'horrible', 'bad', 'terrible', 'awful', 'wow', 'hate']

#Build the word count vector for each review

In [11]:
products['word_count'] = graphlab.text_analytics.count_words(products['review'])
for word in selected_word:
    products[word]= products['word_count'].apply(lambda word_count: word_count[word] if word in word_count.keys() else 0)

In [18]:
products.sort('awesome', ascending=False)
products.head(5)

Columns:
	name	str
	review	str
	rating	float
	word_count	dict
	awesome	int
	great	int
	fantastic	int
	amazing	int
	love	int
	horrible	int
	bad	int
	terrible	int
	awful	int
	wow	int
	hate	int

Rows: 5

Data:
+-------------------------------+-------------------------------+--------+
|              name             |             review            | rating |
+-------------------------------+-------------------------------+--------+
|     Planetwise Wipe Pouch     | it came early and was not ... |  5.0   |
| Annas Dream Full Quilt wit... | Very soft and comfortable ... |  5.0   |
| Stop Pacifier Sucking with... | This is a product well wor... |  5.0   |
| Stop Pacifier Sucking with... | All of my kids have cried ... |  5.0   |
| Stop Pacifier Sucking with... | When the Binky Fairy came ... |  5.0   |
+-------------------------------+-------------------------------+--------+
+-------------------------------+---------+-------+-----------+---------+------+
|           word_count          | awesome | great | fantastic | amazing | love |
+-------------------------------+---------+-------+-----------+---------+------+
| {'and': 3L, 'love': 1L, 'i... |    0    |   0   |     0     |    0    |  1   |
| {'and': 2L, 'quilt': 1L, '... |    0    |   0   |     0     |    0    |  0   |
| {'ingenious': 1L, 'and': 3... |    0    |   0   |     0     |    0    |  2   |
| {'and': 2L, 'parents!!': 1... |    0    |   1   |     0     |    0    |  0   |
| {'and': 2L, 'cute': 1L, 'h... |    0    |   1   |     0     |    0    |  0   |
+-------------------------------+---------+-------+-----------+---------+------+
+----------+-----+----------+-------+-----+------+
| horrible | bad | terrible | awful | wow | hate |
+----------+-----+----------+-------+-----+------+
|    0     |  0  |    0     |   0   |  0  |  0   |
|    0     |  0  |    0     |   0   |  0  |  0   |
|    0     |  0  |    0     |   0   |  0  |  0   |
|    0     |  0  |    0     |   0   |  0  |  0   |
|    0     |  0  |    0     |   0   |  0  |  0   |
+----------+-----+----------+-------+-----+------+
[5 rows x 15 columns]

In [20]:
#Question 1 Answer
for word in selected_word:
    print word, products[word].sum()

 awesome 2002
great 42420
fantastic 873
amazing 1305
love 40277
horrible 659
bad 3197
terrible 673
awful 345
wow 131
hate 1057


In [21]:
#graphlab.canvas.set_target('ipynb')
graphlab.canvas.set_target('browser')

In [23]:
#positive sentiment = 4* or 5* reviews
products['sentiment'] = products['rating'] >=4

In [24]:
products.show()

Canvas is accessible via web browser at the URL: http://localhost:64849/index.html
Opening Canvas in default web browser.


##Let's train the sentiment classifier

In [25]:
train_data,test_data = products.random_split(.8, seed=0)

In [28]:
selected_word_model = graphlab.logistic_classifier.create(train_data, target='sentiment', features=selected_word, validation_set=test_data)

Logistic regression:

--------------------------------------------------------

Number of examples          : 133448

Number of classes           : 2

Number of feature columns   : 11

...

| 4         | 5        | 0.720024     | 0.844269          | 0.843142            |

| 5         | 6        | 0.880036     | 0.844269          | 0.843142            |

| 6         | 7        | 1.048047     | 0.844269          | 0.843142            |

+-----------+----------+--------------+-------------------+---------------------+

SUCCESS: Optimal solution found.

#Evaluate the sentiment model find coefficient

In [31]:
selected_word_model['coefficients'].sort('value')

name,index,class,value,stderr
terrible,None,1,-2.09049998487,0.0967241912229
horrible,None,1,-1.99651800559,0.0973584169028
awful,None,1,-1.76469955631,0.134679803365
hate,None,1,-1.40916406276,0.0771983993506
bad,None,1,-0.985827369929,0.0433603009142
wow,None,1,-0.0541450123333,0.275616449416
great,None,1,0.883937894898,0.0217379527921
fantastic,None,1,0.891303090304,0.154532343591
amazing,None,1,0.892802422508,0.127989503231
awesome,None,1,1.05800888878,0.110865296265


In [32]:
selected_word_model.evaluate(test_data)

{'accuracy': 0.8431419649291376,
 'auc': 0.6648096413721418,
 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      0       |        0        |  234  |
 |      1       |        0        |  130  |
 |      0       |        1        |  5094 |
 |      1       |        1        | 27846 |
 +--------------+-----------------+-------+
 [4 rows x 3 columns],
 'f1_score': 0.914242563530107,
 'log_loss': 0.405474711036565,
 'precision': 0.8453551912568306,
 'recall': 0.9953531598513011,
 'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 100001
 
 Data:
 +-----------+-----+-----+-------+------+
 | threshold | fpr | tpr |   p   |  n   |
 +-----------+-----+-----+-------+------+
 |    0.0    | 1.0 | 1.0 | 27976 | 5328 |
 |   1e-05   | 1.0 | 1.0 | 27976 | 5328 |
 |   2e-05   | 1

In [37]:
diaper_champ_reviews=products[products['name']=='Baby Trend Diaper Champ']
diaper_champ_reviews.head(5)

Columns:
	name	str
	review	str
	rating	float
	word_count	dict
	awesome	int
	great	int
	fantastic	int
	amazing	int
	love	int
	horrible	int
	bad	int
	terrible	int
	awful	int
	wow	int
	hate	int
	sentiment	int

Rows: 5

Data:
+-------------------------+-------------------------------+--------+
|           name          |             review            | rating |
+-------------------------+-------------------------------+--------+
| Baby Trend Diaper Champ | Ok - newsflash.  Diapers a... |  4.0   |
| Baby Trend Diaper Champ | My husband and I selected ... |  1.0   |
| Baby Trend Diaper Champ | Excellent diaper disposal ... |  5.0   |
| Baby Trend Diaper Champ | We love our diaper champ. ... |  5.0   |
| Baby Trend Diaper Champ | Two girlfriends and two fa... |  5.0   |
+-------------------------+-------------------------------+--------+
+-------------------------------+---------+-------+-----------+---------+------+
|           word_count          | awesome | great | fantastic | amazing | love |
+-------------------------------+---------+-------+-----------+---------+------+
| {'just': 2L, 'less': 1L, '... |    0    |   0   |     0     |    0    |  0   |
| {'just': 1L, 'less': 1L, '... |    0    |   0   |     0     |    0    |  0   |
| {'control': 1L, 'am': 1L, ... |    0    |   0   |     0     |    0    |  0   |
| {'and': 3L, 'over.': 1L, '... |    0    |   0   |     0     |    0    |  1   |
| {'just': 1L, 'when': 1L, '... |    0    |   0   |     0     |    0    |  0   |
+-------------------------------+---------+-------+-----------+---------+------+
+----------+-----+----------+-------+-----+------+-----------+
| horrible | bad | terrible | awful | wow | hate | sentiment |
+----------+-----+----------+-------+-----+------+-----------+
|    0     |  0  |    0     |   0   |  0  |  0   |     1     |
|    0     |  0  |    0     |   0   |  0  |  0   |     0     |
|    0     |  0  |    0     |   0   |  0  |  0   |     1     |
|    0     |  0  |    0     |   0   |  0  |  0   |     1     |
|    1     |  0  |    0     |   0   |  0  |  0   |     1     |
+----------+-----+----------+-------+-----+------+-----------+
[5 rows x 16 columns]

In [38]:
selected_word_model.show(view='Evaluation')

Canvas is accessible via web browser at the URL: http://localhost:64849/index.html
Opening Canvas in default web browser.


#Applying the learned model to understand sentiment for Giraffe

In [39]:
diaper_champ_reviews['predicted_sentiment'] = sentiment_model.predict(diaper_champ_reviews, output_type='probability')

##Sort the reviews based on the predicted sentiment and explore

In [42]:
diaper_champ_reviews = diaper_champ_reviews.sort('predicted_sentiment', ascending=False)

In [46]:
diaper_champ_reviews[0]['predicted_sentiment']

0.9984234145936198

##Most positive reviews for the giraffe

In [47]:
diaper_champ_reviews[-1]['predicted_sentiment']

0.32668500817894025

##Show most negative reviews for giraffe

In [36]:
giraffe_reviews[-1]['review']

"My son (now 2.5) LOVED his Sophie, and I bought one for every baby shower I've gone to. Now, my daughter (6 months) just today nearly choked on it and I will never give it to her again. Had I not been within hearing range it could have been fatal. The strange sound she was making caught my attention and when I went to her and found the front curved leg shoved well down her throat and her face a purply/blue I panicked. I pulled it out and she vomited all over the carpet before screaming her head off. I can't believe how my opinion of this toy has changed from a must-have to a must-not-use. Please don't disregard any of the choking hazard comments, they are not over exaggerated!"